In [ ]:
import argparse
import wandb
import os 
import pandas as pd
from src.paths import CODING_DIR, LOGS_DIR
from src.bert.metrics import define_metrics, get_compute_metrics_function
from src.bert.data_prep import (
    get_annotated_answers, get_answer_df, get_classid2trainid,
    get_label2str_dict, prepare_test_dataset, prepare_train_dataset,
    split_llm_train_test, split_train_test_df
)
from src.bert.bert_classifier import BertClassifier
class_mode='coarse'
i=12
answer_df = get_answer_df(class_mode,drop_duplicates=True)
train_df, test_df = split_train_test_df(answer_df, i)
llm_answer_df = get_annotated_answers()
llm_train_df, llm_test_df = split_llm_train_test(llm_answer_df, test_size=0.2)
train_df_combined = pd.concat([train_df, llm_train_df])
test_df_combined = pd.concat([test_df, llm_test_df])

In [7]:
%load_ext autoreload
%autoreload 2

In [10]:
answer_df2 = get_answer_df(class_mode,drop_duplicates=False)


In [ ]:
answer_df2.shape

In [ ]:
answer_df.shape

In [5]:
answer_df['labels_count']= answer_df['labels_list'].apply(lambda x: len(x))

llm_answer_df['labels_count']= llm_answer_df['labels_list'].apply(lambda x: len(x))


In [ ]:
answer_df['labels_count'].mean()


In [ ]:
survey_labels_matrix=  pd.DataFrame(answer_df['labels'].tolist(), columns=label_names )
survey_labels_matrix.drop(list(survey_labels_matrix.filter(regex='LLM refusal')), axis=1, inplace=True)
survey_labels_matrix.sum(axis=0).mean()

In [ ]:
survey_labels_matrix.sum(axis=)

In [ ]:
survey_labels_matrix.sum(axis=0)#.divide(survey_labels_matrix.sum(axis=0).sum())#.value_counts(1)

In [ ]:
llm_labels_matrix.sum(axis=0)#.divide(llm_labels_matrix.sum(axis=0).sum())#.value_counts(1)

In [ ]:
llm_labels_matrix=  pd.DataFrame(llm_answer_df['labels'].tolist(), columns=label_names )
llm_labels_matrix.drop(list(llm_labels_matrix.filter(regex='LLM refusal')), axis=1, inplace=True)
llm_labels_matrix.sum(axis=0).mean()

In [1]:
import src
import numpy as np
from src.paths import MODELS_DIR, OUTPUTS_DIR
import os
import torch
import pandas as pd
from src.data.process_data import process_open_ended, process_wave_data,process_open_ended
from src.data.read_data import load_raw_survey_data, read_stata_file
from src.paths import CODING_DIR, GLES_DIR, PROCESSED_DATA_DIR, ANNOTATED_GENERATIONS_DIR,RAW_DATA_DIR
from src.utils import get_lang
from src.paths import RESULTS_DIR
from tqdm import tqdm



classid2trainid = {int(classname):idx  for idx, classname in enumerate(sorted(pd.read_csv(os.path.join(CODING_DIR,'map.csv')).upperclass_id.unique())) }
df_lookup= pd.read_csv(os.path.join(CODING_DIR,'map.csv'))
label2str= dict(zip(df_lookup.upperclass_id,df_lookup.upperclass_name))
label_names= [label2str[i] for i in range(0,len(label2str)) ]

labels_16= [label_name for label_name in label_names if label_name!='LLM refusal']

In [ ]:
answer_df.groupby('wave')['labels_count'].value_counts()

In [ ]:
from src.analysis.ablationExperiment.utils import get_ablationExperiment_data
survey_labels_dict,llm_labels_dict,survey_population_df,survey_group_pmf,llm_population_df,llm_group_pmf=get_ablationExperiment_data()

In [ ]:
#from src.data.process_data import process_open_ended, process_wave_data,process_open_ended
from src.data.read_data import load_raw_survey_data, read_stata_file

wave_number=12
wave_df, wave_open_ended_df, df_coding_840s = load_raw_survey_data(wave_number)

In [ ]:
df_coding_840s= df_coding_840s.filter(like='840_c', axis=1)
import numpy as np
#replace all cells with nan if it contains str '-'
for m in minus_elts:
    df_coding_840s = df_coding_840s.replace(m,np.nan)

In [ ]:
for i in range(10,22):
    a= df_coding_840s[[f'kp{i}_840_c1',f'kp{i}_840_c2']].dropna(subset=[f'kp{i}_840_c1']).isna().mean()
    print(a)

In [ ]:
minus_elts= [ x  for x in df_coding_840s.kp10_840_c3.unique() if x<0 ]
minus_elts

In [ ]:
answer_df.kpx_840_c2.value_counts(1)
answer_df.kpx_840_c10.value_counts(1)